In [1]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-haiku-4-5"

In [12]:
# Helper functions
def add_user_message(messages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)


def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": text}
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
    }

    if system:
        params["system"] = system

    if stop_sequences:
        params["stop_sequences"] = stop_sequences

    message = client.messages.create(**params)
    return message.content[0].text

In [25]:
import json


def generate_dataset():
    prompt = """
Generate a evaluation dataset for a prompt evaluation. The dataset will be used to evaluate prompts
that generate Python, JSON, or Regex specifically for AWS-related tasks. Generate an array of JSON objects,
each representing task that requires Python, JSON, or a Regex to complete.

Example output:
```json
[
    {
        "task": "Description of task",
        "format": "json" or "python" or "regex",
        "solution_criteria": "Key criteria for evaluating the solution"
    },
    ...additional
]
```

* Focus on tasks that can be solved by writing a single Python function, a single JSON object, or a regular expression.
* Focus on tasks that do not require writing much code

Please generate 3 objects.
"""

    messages = []
    add_user_message(messages, prompt)
    add_assistant_message(messages, "```json")
    text = chat(messages, stop_sequences=["```"])
    return json.loads(text)

In [26]:
dataset = generate_dataset()

dataset

[{'task': 'Parse an AWS CloudWatch log entry and extract the timestamp, log level, and message using a regular expression',
  'format': 'regex',
  'solution_criteria': 'The regex should correctly capture three groups: (1) ISO 8601 timestamp, (2) log level (ERROR, WARN, INFO, DEBUG), and (3) the remaining message text. Should handle variations in timestamp formats.'},
 {'task': 'Write a Python function that takes an AWS S3 object key path and returns a dictionary with the bucket name, folder structure, and file name as separate components',
  'format': 'python',
  'solution_criteria': 'The function should split the S3 path correctly, return a dict with keys for bucket, folders (as list), and filename. Handle edge cases like paths with no folders or trailing slashes.'},
 {'task': 'Create a JSON CloudFormation template snippet for an IAM role that allows an EC2 instance to read objects from a specific S3 bucket',
  'format': 'json',
  'solution_criteria': 'Valid JSON with proper IAM role 

In [27]:
with open("dataset.json", "w") as f:
    json.dump(dataset, f, indent=2)

In [19]:
def run_prompt(test_case):
    """Merges the prompt and test case input, then returns the result"""
    
    prompt = f"""
Please solve the following task:

{test_case["task"]}

* Respond only with Python, JSON, or a plain Regex
* Do not add any comments or commentary or explanation
"""
    
    messages = []
    add_user_message(messages, prompt)
    add_assistant_message(messages, "```code")
    output = chat(messages, stop_sequences=["```"])
    return output

In [28]:
def grade_by_model(test_case, output):

    eval_prompt = f"""
You are an expert AWS code reviewer. Your task is to evaluate the following AI-generated solution.

Original Task:
<task>
{test_case["task"]}
</task>

Solution to Evaluate:
<solution>
{output}
</solution>

Criteria you should use to evaluate the solution:
<criteria>
{test_case["solution_criteria"]}
</criteria>

Output Format
Provide your evaluation as a structured JSON object with the following fields, in this specific order:
- "strengths": An array of 1-3 key strengths
- "weaknesses": An array of 1-3 key areas for improvement
- "reasoning": A concise explanation of your overall assessment
- "score": A number between 1-10

Respond with JSON. Keep your response concise and direct.
Example response shape:
{{
    "strengths": string[],
    "weaknesses": string[],
    "reasoning": string,
    "score": number
}}
    """

    messages = []
    add_user_message(messages, eval_prompt)
    add_assistant_message(messages, "```json")
    eval_text = chat(messages, stop_sequences=["```"])
    return json.loads(eval_text)


In [29]:
import json
import re
import ast


def validate_json(text):
    try:
        json.loads(text.strip())
        return 10
    except json.JSONDecodeError:
        return 0


def validate_python(text):
    try:
        ast.parse(text.strip())
        return 10
    except SyntaxError:
        return 0
    

def validate_regex(text):
    try:
        re.compile(text.strip())
        return 10
    except re.error:
        return 0
    

def grade_syntax(response, test_case):
    format = test_case["format"]
    if format == "json":
        return validate_json(response)
    elif format == "python":
        return validate_python(response)
    else:
        return validate_regex(response)

In [30]:
def run_test_case(test_case):
    """Calls run_prompt, then grades the results"""

    output = run_prompt(test_case)

    model_grade = grade_by_model(test_case, output)
    model_score = model_grade["score"]
    reasoning = model_grade["reasoning"]

    syntax_score = grade_syntax(output, test_case)

    score = (model_score + syntax_score) / 2

    return {
        "output": output,
        "test_case": test_case,
        "score": score,
        "reasoning": reasoning
    }

In [31]:
from statistics import mean

def run_eval(dataset):
    """Loads the dataset and calls run_test_case with each case"""

    results = []

    for test_case in dataset:
        result = run_test_case(test_case)
        results.append(result)

    average_score = mean([result["score"] for result in results])
    print(f"Average score: {average_score}")

    return results

In [32]:
with open("dataset.json", "r") as f:
    dataset = json.load(f)

results = run_eval(dataset)

Average score: 8.5


In [33]:
print(json.dumps(results, indent=2))

[
  {
    "output": "\nimport re\nimport json\n\ndef parse_cloudwatch_log(log_entry):\n    pattern = r'(\\d{4}-\\d{2}-\\d{2}T\\d{2}:\\d{2}:\\d{2}\\.\\d{3}Z)\\s+(\\w+)\\s+(.*)'\n    match = re.match(pattern, log_entry)\n    \n    if match:\n        return {\n            \"timestamp\": match.group(1),\n            \"log_level\": match.group(2),\n            \"message\": match.group(3)\n        }\n    return None\n\nlog_entry = \"2024-01-15T10:30:45.123Z ERROR Connection timeout occurred\"\nresult = parse_cloudwatch_log(log_entry)\nprint(json.dumps(result, indent=2))\n",
    "test_case": {
      "task": "Parse an AWS CloudWatch log entry and extract the timestamp, log level, and message using a regular expression",
      "format": "regex",
      "solution_criteria": "The regex should correctly capture three groups: (1) ISO 8601 timestamp, (2) log level (ERROR, WARN, INFO, DEBUG), and (3) the remaining message text. Should handle variations in timestamp formats."
    },
    "score": 8.0,
 